In [227]:
import pandas as pd
import json

en_de = json.load(open("MLQA_V1/dev/dev-context-en-question-de.json"))
de_en = json.load(open("MLQA_V1/dev/dev-context-de-question-en.json"))

paragraphs_en_de = {}
paragraphs_de_en = {}

for article in en_de["data"]:
    for paragraph in article["paragraphs"]:
        for qa in paragraph["qas"]:
            paragraphs_en_de[qa["id"]] = paragraph

for article in de_en["data"]:
    for paragraph in article["paragraphs"]:
        for qa in paragraph["qas"]:
            paragraphs_de_en[qa["id"]] = paragraph

reverse_en_de = {}
for key, value in paragraphs_en_de.items():
    if value["context"] not in reverse_en_de:
        reverse_en_de[value["context"]] = [key]
    else:
        reverse_en_de[value["context"]].append(key)

reverse_de_en = {}
for key, value in paragraphs_de_en.items():
    if value["context"] not in reverse_de_en:
        reverse_de_en[value["context"]] = [key]
    else:
        reverse_de_en[value["context"]].append(key)

paragraphs_en = []
paragraphs_de = []
for qa in paragraphs_en_de:
    if sorted(reverse_en_de[paragraphs_en_de[qa]["context"]]) != sorted(reverse_de_en[paragraphs_de_en[qa]["context"]]):
        continue
    if paragraphs_en_de[qa] not in paragraphs_en:
        paragraphs_en.append(paragraphs_en_de[qa])
        paragraphs_de.append(paragraphs_de_en[qa])


questions = []

for paragraph in paragraphs_en:
    for qa in paragraph["qas"]:
        questions.append({"id": qa["id"], "question_en": qa["question"], "answer_en": qa["answers"][0]["text"], "paragraph_en": paragraph["context"]})

df = pd.DataFrame(questions)

for paragraph in paragraphs_de:
    for qa in paragraph["qas"]:
        df.loc[df["id"] == qa["id"], "question_de"] = qa["question"]
        df.loc[df["id"] == qa["id"], "answer_de"] = qa["answers"][0]["text"]
        df.loc[df["id"] == qa["id"], "paragraph_de"] = paragraph["context"]


4054